In [1]:
# 为环境变量做好准备
import os
from io import StringIO
from dotenv import load_dotenv

env_text = """
HOME=./eztransfer_modelzoo
"""

load_dotenv(stream=StringIO(env_text), override=True, verbose=True)
os.environ.get("HOME")

'./eztransfer_modelzoo'

In [2]:
# 让本地的 easytransfer 的优先级更高
import sys
sys.path.insert(0, os.path.abspath('../'))
sys.path

['d:\\code\\github\\EasyTransfer',
 'd:\\code\\github\\EasyTransfer\\examples',
 'c:\\Users\\tzh\\.vscode\\extensions\\ms-toolsai.jupyter-2021.11.1001550889\\pythonFiles\\vscode_datascience_helpers',
 'c:\\Users\\tzh\\.vscode\\extensions\\ms-toolsai.jupyter-2021.11.1001550889\\pythonFiles',
 'c:\\Users\\tzh\\.vscode\\extensions\\ms-toolsai.jupyter-2021.11.1001550889\\pythonFiles\\lib\\python',
 'C:\\Anaconda3\\envs\\tf2\\python39.zip',
 'C:\\Anaconda3\\envs\\tf2\\DLLs',
 'C:\\Anaconda3\\envs\\tf2\\lib',
 'C:\\Anaconda3\\envs\\tf2',
 '',
 'C:\\Anaconda3\\envs\\tf2\\lib\\site-packages',
 'C:\\Anaconda3\\envs\\tf2\\lib\\site-packages\\win32',
 'C:\\Anaconda3\\envs\\tf2\\lib\\site-packages\\win32\\lib',
 'C:\\Anaconda3\\envs\\tf2\\lib\\site-packages\\Pythonwin']

In [3]:
# 导入 easytransfer, 并验证, 因为原始的没有 __version__ 这个参数
import importlib
import easytransfer as easytransfer
importlib.reload(easytransfer)
print(easytransfer.__version__)

import tensorflow as tf

INFO:tensorflow:*********** tf.__version__ is 2.7.0 ******
1.0.0


In [4]:
# jupyter 特殊操作
old_argv = sys.argv
print(old_argv)
sys.argv = old_argv[:1]

['ipykernel_launcher', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"28869bbf-4e65-4636-a638-fbf3b32dc081"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=C:\\Users\\tzh\\AppData\\Local\\Temp\\tmp-8760buZ5JuhzxLln.json']


In [5]:
from easytransfer.datasets import CSVReader

In [6]:
reader = CSVReader("D:/code/py_nlp_classify/data/small/train.csv", is_training=True, input_schema="label:str:1,content:str:1", batch_size=4)
reader

INFO:tensorflow:num_parallel_batches 1
INFO:tensorflow:shuffle_buffer_size None
INFO:tensorflow:prefetch_buffer_size 1
INFO:tensorflow:batch_size 4
INFO:tensorflow:distribution_strategy None
INFO:tensorflow:num_micro_batches 1
INFO:tensorflow:input_schema label:str:1,content:str:1
INFO:tensorflow:D:/code/py_nlp_classify/data/small/train.csv, total number of training examples 1000


In [7]:
dataset = reader.get_dataset()
dataset

INFO:tensorflow:Random shuffle on the whole 1000 training examples


<PrefetchDataset shapes: {label: (None,), content: (None,)}, types: {label: tf.string, content: tf.string}>

In [8]:
next(iter(dataset))

{'label': <tf.Tensor: shape=(4,), dtype=string, numpy=array([b'game', b'house', b'car', b'world'], dtype=object)>,
 'content': <tf.Tensor: shape=(4,), dtype=string, numpy=
 array([b'\xe5\x85\xa8\xe6\xb0\x91k\xe6\xad\x8c\xe5\xbd\x95\xe9\x9f\xb3\xe6\x97\xb6\xe4\xbc\x9a\xe5\x87\xba\xe7\x8e\xb0\xe5\x8e\x9f\xe5\xa3\xb0\xe5\xba\x94\xe8\xaf\xa5\xe6\x80\x8e\xe4\xb9\x88\xe5\x8a\x9e\xef\xbc\x9f',
        b'\xe5\x86\x9c\xe6\x9d\x91\xe6\x89\xbf\xe5\x8c\x85\xe5\x9c\x9f\xe5\x9c\xb0100\xe4\xba\xa9\xef\xbc\x8c\xe5\x9b\xbd\xe5\xae\xb6\xe6\x9c\x89\xe4\xbb\x80\xe4\xb9\x88\xe8\xa1\xa5\xe8\xb4\xb4\xef\xbc\x9f',
        b'\xe5\xbe\xb7\xe7\xb3\xbb\xe5\xa4\xa7\xe6\x88\x98\xe7\xbe\x8e\xe7\xb3\xbb\xef\xbc\x8c\xe6\x96\xaf\xe6\x9f\xaf\xe8\xbe\xbe\xe6\x9f\xaf\xe7\xb1\xb3\xe5\x85\x8b\xe5\xaf\xb9\xe6\xa0\x87\xe5\x88\xab\xe5\x85\x8b\xe6\x98\x82\xe7\xa7\x91\xe6\x8b\x89\xef\xbc\x8c\xe8\xb0\x81\xe6\x9b\xb4\xe5\x8f\x97\xe5\xb9\xb4\xe8\xbd\xbb\xe4\xba\xba\xe6\xac\xa2\xe8\xbf\x8e SUV,\xe6\x9f\xaf\xe7\xb1\xb3\xe5\x85\x8b,\x

In [9]:
next(iter(dataset)).keys()

dict_keys(['label', 'content'])

In [10]:
# 加载模型
from easytransfer.model_zoo.modeling_bert import BertConfig, BertPreTrainedModel, MyBertPreTrainedModel
bert_config_path = "D:/code/py_nlp_classify/model/bert/google-bert-base-zh/config.json"
bert_config = BertConfig.get(bert_config_path)
my_bert_model = MyBertPreTrainedModel(bert_config)
# 原来是要调用过后才生成 weights
out_put = my_bert_model(my_bert_model.dummy_inputs(512), mode="eval", output_features=False)
archive_file = "D:/code/py_nlp_classify/model/bert/google-bert-base-zh/bert_model.ckpt"
my_bert_model._init_from_pretrained_model(archive_file)

# my_bert_model(my_bert_model.dummy_inputs(512), mode="eval", output_features=False)

input_shape (1, 512)
input_shape [1, 512]
从文件中还原预训练模型
INFO:tensorflow:Load weights from D:/code/py_nlp_classify/model/bert/google-bert-base-zh/bert_model.ckpt


In [11]:
import json
from easytransfer import preprocessors, Config
config_json = json.load(open("user_config_tf2_test.json", "r", encoding="utf-8"))
config = Config(mode="train_and_evaluate_on_the_fly", config_json=config_json)
preprocessor = preprocessors.get_preprocessor(
    "D:/code/py_nlp_classify/model/bert/google-bert-base-zh/bert_model.ckpt", user_defined_config=config
)

INFO:tensorflow:***************** modelZooBasePath ./eztransfer_modelzoo\.eztransfer_modelzoo ***************


In [12]:
process_out = preprocessor(next(iter(dataset)))
process_out

([<tf.Tensor: shape=(4, 64), dtype=int64, numpy=
  array([[ 101, 1963, 3362,  872, 3221,  671, 1399, 5439, 2360, 8024,  872,
          2190, 4408,  677, 4638, 2345, 4495, 3300,  784,  720, 6206, 3724,
          8043,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0],
         [ 101,  671, 3340, 2544,  928, 1440, 6401,  872, 1762, 2813, 2336,
          2582,  720, 1391, 1600, 4381,  727, 6579, 8013, 4125, 6862, 3119,
          5966, 8013,  691, 1068, 6125,  117, 1920, 4215, 2397,  692,  117,
          2476, 1294, 1409,  117, 4607, 6205, 3959,  117,  702, 1736,  102,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 3125,  752, 8038, 3330, 5439, 3727, 3612, 717

In [13]:
print(type(process_out))
print(len(process_out))

print(len(process_out[0]))
print(process_out[1])

<class 'tuple'>
2
3
tf.Tensor(
[[ 8]
 [10]
 [13]
 [ 2]], shape=(4, 1), dtype=int64)


In [14]:
dataset = reader.get_dataset(preprocessor)
next(iter(dataset))

Instructions for updating:
delimiter is deprecated, please use sep instead.


(<tf.Tensor: shape=(3, 4, 64), dtype=int64, numpy=
 array([[[  101,  4692,  5401,  1196,  2110,  5739,  6427,  8271,   758,
           3299,  6841,  1196,  2900,  1298,  6963,  2356,  1957,  2111,
            117,  1318,  4886,   117,  8450, 10712, 10936,   117,  4868,
           2968,  1909,  3821,  1046,   117,  5294,  5276,  1957,  2094,
           1745,  7063,   117,  3654,  6854,  1398,  2495,   117, 10373,
            117,  3617,  3307,  6963,  2356,   117,  3449,  2209,  5384,
           3172,   117,  7946,  7178,  5526,  1765,   117,  1628,  6117,
            102],
         [  101,  2798,  6371,  6399,   697,  1453,  4638,  4685,   779,
           4511,  4960,  4197,   779,   749,   677,  3341,  8024,  2867,
           5318,  1400,  8024,   800,  4994,  4197,  2828,  2769,  2861,
           7946,   749,  8043,  8013,  4685,   779,   117,  1062,  1218,
           1447,   117,  4685,   779,  4511,   117,  4510,  2512,   102,
              0,     0,     0,     0,     0,     0,    

In [15]:
from easytransfer import layers
import tensorflow.keras as keras
inputs = tf.keras.Input(shape=(None, None,), dtype=tf.int64)

dense = layers.Dense(config.num_labels, kernel_initializer=layers.get_initializer(0.02), name="dense")
_, pooled_output = my_bert_model(inputs)
output = dense(pooled_output)
my_model = keras.Model(inputs=inputs, outputs=output)

input_shape (None, None, None)
input_shape [<tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_4:0' shape=() dtype=int32>]


In [16]:
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None)]      0         
                                                                 
 my_bert_pre_trained_model (  ((None, None, 768),      102882442 
 MyBertPreTrainedModel)       (None, 768))                       
                                                                 
 dense (Dense)               (None, 15)                11535     
                                                                 
Total params: 102,893,977
Trainable params: 102,893,977
Non-trainable params: 0
_________________________________________________________________


In [17]:
from easytransfer.optimizers.adam_weight_decay_optimizer import AdamWeightDecayOptimizer

my_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # optimizer=AdamWeightDecayOptimizer(
    #     learning_rate=1e-5,
    #     weight_decay_rate=0,
    #     beta_1=0.9,
    #     beta_2=0.999,
    #     epsilon=1e-6,
    #     # 有几个层不使用权重衰减系数
    #     exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"],
    # ),
    optimizer=keras.optimizers.Adam(
        learning_rate=1e-5,
        beta_1=0.9,
        beta_2=0.99,
        epsilon=1e-6,
    ),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [18]:
for x in next(iter(dataset))[0]:
    print(type(x), x.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'> (4, 64)
<class 'tensorflow.python.framework.ops.EagerTensor'> (4, 64)
<class 'tensorflow.python.framework.ops.EagerTensor'> (4, 64)


In [19]:
eval_reader = CSVReader("D:/code/py_nlp_classify/data/small/dev.csv", is_training=True, input_schema="label:str:1,content:str:1", batch_size=4)
eval_dataset = eval_reader.get_dataset(preprocessor)
my_model.fit(x=dataset, epochs=5, validation_data=eval_dataset)

INFO:tensorflow:num_parallel_batches 1
INFO:tensorflow:shuffle_buffer_size None
INFO:tensorflow:prefetch_buffer_size 1
INFO:tensorflow:batch_size 4
INFO:tensorflow:distribution_strategy None
INFO:tensorflow:num_micro_batches 1
INFO:tensorflow:input_schema label:str:1,content:str:1
INFO:tensorflow:D:/code/py_nlp_classify/data/small/dev.csv, total number of training examples 100
INFO:tensorflow:Random shuffle on the whole 100 training examples
Epoch 1/5
input_shape (3, None, 64)
input_shape [<tf.Tensor 'model/my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, 64]
input_shape (3, None, 64)
input_shape [<tf.Tensor 'model/my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, 64]
    250/Unknown - 38s 109ms/step - loss: 2.2736 - sparse_categorical_accuracy: 0.3240input_shape (3, None, 64)
input_shape [<tf.Tensor 'model/my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, 64]
250/250 [==============================] - 41s 123ms/step

In [20]:
my_model.evaluate(eval_dataset)

25/25 [==============================] - 1s 43ms/step - loss: 1.4210 - sparse_categorical_accuracy: 0.5900


[1.421048879623413, 0.5899999737739563]

In [21]:
my_model.save("my_model")

input_shape (None, None, None)
input_shape [<tf.Tensor 'model/my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'model/my_bert_pre_trained_model/bert/strided_slice_4:0' shape=() dtype=int32>]
input_shape (None, None, None)
input_shape [<tf.Tensor 'bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'bert/strided_slice_4:0' shape=() dtype=int32>]
input_shape (None, None, None)
input_shape [<tf.Tensor 'bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'bert/strided_slice_4:0' shape=() dtype=int32>]
input_shape (None, None, None)
input_shape [<tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_4:0' shape=() dtype=int32>]
input_shape (None, None, None)
input_shape [<tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_3:0' shape=() dtype=int32>, <tf.Tensor 'my_bert_pre_trained_model/bert/strided_slice_4:0' shape=() dtype=int32>]
input_shape (None, Non

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [22]:
new_model = keras.models.load_model("my_model")

In [23]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None)]      0         
                                                                 
 my_bert_pre_trained_model (  ((None, None, 768),      102882442 
 MyBertPreTrainedModel)       (None, 768))                       
                                                                 
 dense (Dense)               (None, 15)                11535     
                                                                 
Total params: 102,893,977
Trainable params: 102,893,977
Non-trainable params: 0
_________________________________________________________________


In [24]:
new_model.evaluate(eval_dataset)

25/25 [==============================] - 2s 42ms/step - loss: 1.4210 - sparse_categorical_accuracy: 0.5900


[1.421048879623413, 0.5899999737739563]